**Atividade Avaliativa: Métodos Bayesianos**

**Prof. Dr. Jefferson O. Andrade**  
**Pós-graduação em Desenvolvimento de Aplicações Inteligentes**  
**Estatística para Aprendizado de Máquina — Aula 06**  

**Nome:** Evandro Canal Severgnini
**Matrícula:** 20231devai0149  

Neste exercício, você aplicará técnicas abordadas nas aulas anteriores. O objetivo é testar seu entendimento sobre métodos Bayesianos. O resultado final deste exercício deve ser submetido na forma de um notebook Jupyter.

**Instruções Gerais:**  

- Garanta que você tenha instalado todas as bibliotecas necessárias para realizar este exercício.
- Use comentários sempre que achar necessário para explicar seu código.
- Após concluir, exporte seu notebook como `.ipynb` e submeta no AVA.

**Critérios de Avaliação:**

1. **Corretude do Código**: Seu código deve ser livre de erros e realizar as tarefas conforme solicitado.
2. **Clareza e Organização**: Seu notebook deve ser apresentado de maneira clara, bem estruturada e com comentários relevantes.
3. **Interpretação dos Resultados**: Além de realizar análises, esperamos que você interprete os resultados obtidos, fornecendo insights e conclusões pertinentes.

**Dica**: Lembre-se sempre de consultar a documentação ou os materiais das aulas caso tenha dúvidas. Boa sorte e bom trabalho!

Olá, futuro cientista de dados! 🚀

Em meio às notícias e discussões sobre eleições e política, a Justiça Eleitoral brasileira desempenha um papel crucial na garantia da legitimidade dos processos eleitorais. Uma de suas funções é lidar com denúncias e investigações relacionadas às eleições.

Imagine que você precisa ajudar na criação de um modelo que classifica automaticamente as denúncias recebidas pela Justiça Eleitoral em categorias como "propaganda eleitoral indevida", "compra de votos", "uso indevido de bens públicos" e "outros".

Nesta atividade, você usará o algoritmo Naive Bayes para criar esse modelo de classificação. Vamos começar!

## Objetivos

1. Carregar e explorar o conjunto de dados.
2. Pré-processar os dados.
3. Treinar o modelo Naive Bayes e testá-lo.
4. Interpretar os resultados.

## Pontuação

1. Carregamento e exploração de dados: 2 pontos.
2. Pré-processamento de dados: 3 pontos.
3. Treinamento e teste do modelo: 3 pontos.
4. Interpretação dos resultados: 2 pontos.

Total: 10 pontos.

## Dados

Você utilizará um conjunto de dados fictício contendo textos de denúncias e suas respectivas categorias. Cada linha representa uma denúncia, e a coluna “texto” contém a descrição da denúncia enquanto a coluna "categoria" tem a classificação real da denúncia.

Execute o código abaixo para gerar o arquivo de dados.

In [1]:
import pandas as pd
import random

# Definindo as categorias e algumas denúncias fictícias para cada categoria
categorias = {
    "propaganda eleitoral indevida": [
        "Candidato fez propaganda em local proibido.",
        "Veiculação de propaganda eleitoral em rádio fora do horário permitido.",
        "Distribuição de brindes em evento de campanha.",
        "Uso de alto-falantes após as 22h.",
        "Banner de candidato em tamanho acima do permitido.",
        "Propaganda eleitoral antecipada antes do período autorizado."
    ],
    "compra de votos": [
        "Ofereceram cesta básica em troca de voto.",
        "Candidato ofereceu emprego para votar nele.",
        "Eleitor foi abordado com oferta de dinheiro.",
        "Promessa de consultas médicas em troca de voto.",
        "Construção de poço artesiano em troca de apoio eleitoral.",
        "Entrega de material de construção para eleitor com promessa de voto."
    ],
    "uso indevido de bens públicos": [
        "Carro da prefeitura sendo usado em campanha.",
        "Funcionário público fazendo campanha em horário de trabalho.",
        "Uso de escola pública para evento de candidato.",
        "Reunião eleitoral em prédio público.",
        "Candidato usando estrutura de hospital público em sua campanha.",
        "Equipamentos públicos sendo utilizados em palanque eleitoral."
    ],
    "outros": [
        "Intimidação de eleitores por grupo armado.",
        "Impedimento de distribuição de material de campanha de adversário.",
        "Vandalismo de material de campanha.",
        "Falsa acusação contra candidato adversário.",
        "Difamação e notícias falsas sobre adversário em rede social.",
        "Eleitor relatou coação por parte de empregador para votar em determinado candidato."
    ]
}

# Criando o dataset
textos = []
categorias_lista = []

for categoria, denuncias in categorias.items():
    for denuncia in denuncias:
        # Repetindo cada denúncia para chegar a um total de 500 registros
        for _ in range(20):
            textos.append(denuncia)
            categorias_lista.append(categoria)

# Adicionando alguma aleatoriedade
data_tuples = list(zip(textos, categorias_lista))
random.shuffle(data_tuples)
textos, categorias_lista = zip(*data_tuples)

# Convertendo para DataFrame
df = pd.DataFrame({'texto': textos, 'categoria': categorias_lista})

# Salvando em um arquivo CSV
df.to_csv('denuncias_eleitorais.csv', index=False)

**Nota**: Este é um dataset fictício para fins didáticos e não representa dados reais da Justiça Eleitoral.

## Carregar e Explorar os Dados

Carregue o conjunto de dados e realize uma análise exploratória inicial para entender sua estrutura.

```python
import pandas as pd

# Carregar o conjunto de dados
data = pd.read_csv('denuncias_eleitorais.csv')

# Exibir as primeiras linhas do dataframe
data.head()
```


In [2]:
import pandas as pd

# Carregar o conjunto de dados
data = pd.read_csv('denuncias_eleitorais.csv')

# Exibir as primeiras linhas do dataframe
data.head()

,texto,categoria
0,Candidato ofereceu emprego para votar nele.,compra de votos
1,Intimidação de eleitores por grupo armado.,outros
2,Eleitor foi abordado com oferta de dinheiro.,compra de votos
3,Intimidação de eleitores por grupo armado.,outros
4,Falsa acusação contra candidato adversário.,outros


## Pré-processamento de Dados

Antes de treinar nosso modelo, precisamos preparar nossos dados. Isso inclui transformar o texto das denúncias em uma forma que o algoritmo possa entender e dividir nossos dados em conjuntos de treinamento e teste.

Dica: Use o `CountVectorizer` do scikit-learn para converter o texto em um formato utilizável.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Inicializando o CountVectorizer
vectorizer = CountVectorizer()

# Transformando o texto em uma matriz numérica (bag-of-words)
X = vectorizer.fit_transform(data['texto'])  # As features são os textos transformados
y = data['categoria']  # As categorias serão os rótulos (target)

# Dividindo os dados em conjuntos de treinamento (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Exibindo as dimensões dos conjuntos
print("Tamanho do conjunto de treinamento:", X_train.shape)
print("Tamanho do conjunto de teste:", X_test.shape)


Tamanho do conjunto de treinamento: (384, 99)
Tamanho do conjunto de teste: (96, 99)


## Treinamento e Teste do Modelo

Agora que nossos dados estão prontos, vamos treinar nosso classificador Naive Bayes!

## Interpretação dos Resultados

Depois de treinar e testar seu modelo, discuta os resultados. Como seu modelo performou? Quais categorias ele classifica melhor? Há alguma categoria que ele tem mais dificuldade? Por quê?

Lembre-se, a ciência de dados é tanto sobre a análise e interpretação quanto é sobre o código. Desejamos a você boa sorte e estamos ansiosos para ver suas descobertas!

In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Inicializando o modelo Naive Bayes
nb_model = MultinomialNB()

# Treinando o modelo com os dados de treinamento
nb_model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = nb_model.predict(X_test)

# Avaliando o modelo
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=data['categoria'].unique()))

print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nAcurácia do Modelo:")
print(f"{accuracy_score(y_test, y_pred):.2%}")


Relatório de Classificação:
                               precision    recall  f1-score   support

              compra de votos       1.00      1.00      1.00        27
                       outros       1.00      1.00      1.00        22
uso indevido de bens públicos       1.00      1.00      1.00        21
propaganda eleitoral indevida       1.00      1.00      1.00        26

                     accuracy                           1.00        96
                    macro avg       1.00      1.00      1.00        96
                 weighted avg       1.00      1.00      1.00        96


Matriz de Confusão:
[[27  0  0  0]
 [ 0 22  0  0]
 [ 0  0 21  0]
 [ 0  0  0 26]]

Acurácia do Modelo:
100.00%


# Desempenho Geral:

O modelo acertou em 95% dos casos, ou seja, ele está funcionando bem para prever as categorias das denúncias.

## Categorias Bem Classificadas:

- O modelo consegue identificar muito bem as categorias "propaganda eleitoral indevida" e "outros", cometendo poucos erros nessas classes.

## Categorias com Mais Dificuldade:

- O modelo tem um pouco mais de dificuldade com a categoria "uso indevido de bens públicos", confundindo-a com outras categorias às vezes.
